In [1]:
import pandas as pd
from lib import search_display, search_df

/home/coder/ucsb-writing-data/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# import reflections data
df = pd.read_csv("data/reflections.csv")

## pre-filtering: query a subset of the reflections (optional)
#
## example: subset df to students from a specific department
# df = df[ df.DEPT0 == "EEMB/MCDB"]
#
## example: subset df to first gen students in a specific department
# df = df[ (df.DEPT0 == "EEMB/MCDB") & (df.first_gen_coll_student == 1)]

In [3]:
# search using mxbai and display 10 results
search_display("not confident with my writing skills", df, limit=10, model="mxbai")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(distance: 10.4, perm: Student601) I am not at all confident with my writing abilities. I feel that I have the knowledge and practice to be able to read texts and analyze them but when It comes down to writing and putting my thoughts into a worded format that is where I struggle. I have always excelled when it comes to math but I consistently fall short when it comes into comparison with my English. I feel that I need more time/teaching/review to be on par with college writing standards.

(distance: 10.7, perm: Student2118) I've always had a difficult time writing and expressing my thoughts through words. Especially when it comes to storytelling, I'm just not fully confident in my writing skills. I look back at my previous writing and I only feel confident in two essays I've written. With my attention span, I also don't tend to enjoy reading, because I have a difficult time comprehending some texts and end up reading without consuming the information presented. I want to get serious about my writing and become better, so I just don't see myself starting off with writing 2. I think that would just stress me out and discourage me. 

(distance: 11.0, perm: Student2135) I think I am not prepared to write the text like this. I don't think I have a perfect writing skills to complete a paragraph like that

(distance: 11.3, perm: Student1216) Although I have learned how to become a better writer in high school, I am not as comfortable and confident as I hope to be in writing. It takes me a lot of time and extra help to write a decent paper. It is doable but I believe that extra practice will do me good. I believe that I should take every opportunity to improve my writing to feel confident and comfortable in doing any writing assignment.  

...

(distance: 11.3, perm: Student3041) I think that while I have been confident with my writing skills in the past, I would want something that would help me ease into college level work. It would definitely make me more comfortable and give me a better transition. It would also help me become more confident with the work that I produce as college student.

(distance: 11.3, perm: Student833) I have done a few responses to documents and speeches by writing my own analysis papers, but I do not feel extremely confident on the process or on my ability to achieve all of an assignments criteria with the knowledge that I have now.

(distance: 11.4, perm: Student2227) I think that my writing and english skills would need some refining and I am not as confident to take writing 2.

(distance: 11.4, perm: Student1170) The lack of writing and reading over the past year and a half has made me doubt my abilities and feel less confident with my writing. I do think I'd be able to handle Writing 2, but I would feel more comfortable starting with readings and writings that I've seen and done in the past. 

(distance: 11.5, perm: Student2871) Though I have had experience with most of the course material in Writing 1, I am still not confident with my writing as I've stated that I have trouble putting thoughts to pen. I would like to push myself and start with Writing 2, but I fear that I may not be able to keep up with my peers. I hope that if I am  to be placed in Writing 1, that I would get the feedback I need to progress at an acceptable level.

In [4]:
# same query, but using normalized mxbai
search_display("not confident with my writing skills", df, limit=10, model="mxbai_norm")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(distance: 0.58, perm: Student601) I am not at all confident with my writing abilities. I feel that I have the knowledge and practice to be able to read texts and analyze them but when It comes down to writing and putting my thoughts into a worded format that is where I struggle. I have always excelled when it comes to math but I consistently fall short when it comes into comparison with my English. I feel that I need more time/teaching/review to be on par with college writing standards.

(distance: 0.60, perm: Student2118) I've always had a difficult time writing and expressing my thoughts through words. Especially when it comes to storytelling, I'm just not fully confident in my writing skills. I look back at my previous writing and I only feel confident in two essays I've written. With my attention span, I also don't tend to enjoy reading, because I have a difficult time comprehending some texts and end up reading without consuming the information presented. I want to get serious about my writing and become better, so I just don't see myself starting off with writing 2. I think that would just stress me out and discourage me. 

(distance: 0.62, perm: Student2135) I think I am not prepared to write the text like this. I don't think I have a perfect writing skills to complete a paragraph like that

(distance: 0.64, perm: Student2227) I think that my writing and english skills would need some refining and I am not as confident to take writing 2.

...

(distance: 0.65, perm: Student3041) I think that while I have been confident with my writing skills in the past, I would want something that would help me ease into college level work. It would definitely make me more comfortable and give me a better transition. It would also help me become more confident with the work that I produce as college student.

(distance: 0.65, perm: Student2527) I am not confident enough in my abilities to succeed in Writing 2, I think that the added support of taking Writing 1, it would benefit me more than immediately going into Writing 2. I have struggled in the past when it comes to reading certain fictional texts and understanding the authors meaning and thereby making it difficult for me to write full essays analyzing the authors purpose or compare then to other texts. I am confident in my ability in reading any sort of non-fiction text and writing any sort of essay on them. 

(distance: 0.66, perm: Student1216) Although I have learned how to become a better writer in high school, I am not as comfortable and confident as I hope to be in writing. It takes me a lot of time and extra help to write a decent paper. It is doable but I believe that extra practice will do me good. I believe that I should take every opportunity to improve my writing to feel confident and comfortable in doing any writing assignment.  

(distance: 0.66, perm: Student2871) Though I have had experience with most of the course material in Writing 1, I am still not confident with my writing as I've stated that I have trouble putting thoughts to pen. I would like to push myself and start with Writing 2, but I fear that I may not be able to keep up with my peers. I hope that if I am  to be placed in Writing 1, that I would get the feedback I need to progress at an acceptable level.

(distance: 0.66, perm: Student833) I have done a few responses to documents and speeches by writing my own analysis papers, but I do not feel extremely confident on the process or on my ability to achieve all of an assignments criteria with the knowledge that I have now.

In [6]:
# output full table (results only include responses from firsts gen students)
result = search_df("rhetorical analysis", df[df.first_gen_coll_student == 1], model="mxbai")
# print first few rows
result.head()

,perm,survey_50,HSGPA,student_selection,final_placement,auto_read,cwp_quarter,data_permission,language,PR_quarter,...,EOPGRP,Gender,ETH_GRP,ETH,DEPT0,first_gen_coll_student,embedding,result_question_id,result_distance,result_text
0,Student2420,48.0,4.21,W2,W2,auto,M1-22,My data collected as part of this project may ...,I am most comfortable using English.,None,...,NOT_EOP,F,4,Asian,Psychological and Brain Sciences,1,"[0.017748397, -0.0064662546, 0.04986563, 0.005...",r2,0.565852,In my high school writing courses we heavily p...
1,Student3238,42.0,4.16,W2,W2,auto,M2-23,My data collected as part of this project may ...,I am most comfortable using English.,None,...,EOP,F,4,Asian,College of Letters and Science,1,"[0.021607453, -0.0077467826, 0.023134658, 0.02...",r2,0.591458,"The ""Rhetorical Analysis"" assignment has a res..."
2,Student1602,32.0,3.58,W1,W1,auto,M2-21,My data collected as part of this project may ...,I am most comfortable using English.,None,...,NOT_EOP,F,3,Latino,EEMB/MCDB,1,"[0.030501746, -0.00709839, 0.026211325, -0.005...",r2,0.600676,This assignment Rhetorical Analysis is similar...
3,Student2606,35.0,NaN,W1,W1,auto,W21,My data collected as part of this project may ...,didn't ask,None,...,EOP,F,4,Asian,EEMB/MCDB,1,"[-0.006085272, -0.0178766, 0.039734654, -0.017...",r2,0.606979,In high school I have done rhetorical analysis...
4,Student176,43.0,3.87,W2,W2,read,F22,My data collected as part of this project may ...,I am most comfortable using English.,None,...,EOP,M,1,African American,Sociology,1,"[0.024260724, -0.0118631, 0.03522913, 0.005190...",r2,0.609220,I worked on multiple Rhetorical Analysis assig...
